<a href="https://colab.research.google.com/github/ShanmukhiKairuppala/Diabetic_Retinopathy_Detection/blob/main/Mini_Project_DR_Flask_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install Flask ngrok scikit-image tensorflow


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 8.7 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, img_as_ubyte
from tensorflow.keras.models import load_model
import cv2
from flask import Flask, request, jsonify

app = Flask(__name__)
model = load_model('/content/drive/MyDrive/FineTunedModel')

def preprocess_image(image, size=(256, 256)):
    if image.ndim == 2 or image.max() == image.min():
        print("Skipping image: Image is either grayscale or completely black.")
        return None

    if image.ndim == 3 and image.shape[2] == 3:
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        image = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    elif image.ndim == 2:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        image = clahe.apply(image)

    resized_image = cv2.resize(image, size)
    return img_as_ubyte(resized_image)

def augment_image(image, base_filename, output_dir, retinopathy=True):
    transformations = [image]
    if retinopathy:
        for angle in [90, 120, 180, 270]:
            rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
            transformations.append(rotated_image)
    mirrored_image = cv2.flip(image, 1)
    transformations.append(mirrored_image)
    augmented_filenames = []
    for idx, transformed_image in enumerate(transformations):
        transformed_filename = f"{base_filename}_{idx}.png"
        transformed_path = os.path.join(output_dir, transformed_filename)
        io.imsave(transformed_path, transformed_image)
        augmented_filenames.append(transformed_filename)
    return augmented_filenames

def preprocess_and_augment_single_image(image_path, output_dir, base_filename):
    image = io.imread(image_path)
    preprocessed_image = preprocess_image(image)
    if preprocessed_image is not None:
        augmented_filenames = augment_image(preprocessed_image, base_filename, output_dir, True)
        augmented_images = [io.imread(os.path.join(output_dir, filename)) for filename in augmented_filenames]
        return preprocessed_image, augmented_images
    return None, []

def predict_and_display(model, image_path):
    temp_dir = '/content/temp'
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    preprocessed_image, augmented_images = preprocess_and_augment_single_image(image_path, temp_dir, 'new_image')

    if preprocessed_image is not None:
        image_array = np.expand_dims(preprocessed_image / 255.0, axis=0)
        prediction = model.predict(image_array)
        predicted_class = np.argmax(prediction)
        dr_levels = ["No DR", "Mild DR", "Moderate DR", "Severe DR", "Proliferative DR"]

        num_images = 1 + len(augmented_images)
        plt.figure(figsize=(15, 5))

        plt.subplot(1, num_images, 1)
        plt.imshow(preprocessed_image)
        plt.title('Input Image')

        for i, augmented_image in enumerate(augmented_images):
            plt.subplot(1, num_images, i + 2)
            plt.imshow(augmented_image)
            plt.title(f'Augmented {i + 1}')

        plt.suptitle(f'Predicted DR Level: {predicted_class} ({dr_levels[predicted_class]})')
        result_path = os.path.join(temp_dir, 'result.png')
        plt.savefig(result_path)
        plt.close()

        return result_path, preprocessed_image, augmented_images, predicted_class
    else:
        return None, None, [], None

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['file']
    file_path = '/content/' + file.filename
    file.save(file_path)
    result_path, preprocessed_image, augmented_images, predicted_class = predict_and_display(model, file_path)
    if result_path:
        return jsonify({
            'result_image': result_path,
            'preprocessed_image': preprocessed_image.tolist(),
            'augmented_images': [img.tolist() for img in augmented_images],
            'predicted_class': predicted_class
        })
    else:
        return jsonify({'error': 'Image preprocessing failed'}), 400

if __name__ == '__main__':
    from google.colab.output import eval_js
    from IPython.display import Javascript

    print("Running app on Colab...")

    get_ipython().system_raw('ngrok http 5000 &')
    get_ipython().system_raw('python3 main.py &')

    ngrok_url = eval_js("google.colab.kernel.proxyPort(5000)")

    print(f"Open the following link in your browser: {ngrok_url}")


Running app on Colab...
Open the following link in your browser: https://h2mpbvbj4jm-496ff2e9c6d22116-5000-colab.googleusercontent.com/
